# **PROJET LEYENDA - LIVRABLE 2 : TRAITEMENT D'IMAGE**

   ## **INTRODUCTION**

Dans le cadre du projet TouNum, l'objectif est de créer une solution entièrement automatisée pour analyser et générer des légendes d'images (captioning).
Une étape essentielle pour atteindre cet objectif est d'améliorer la qualité des images afin de les rendre plus adaptées aux tâches ultérieures.

La qualité des images peut avoir un impact significatif sur la précision des modèles de classification et de génération de légendes, en particulier lorsqu'on travaille avec des données bruitées ou de faible qualité.
Par conséquent, une phase de prétraitement efficace des images est indispensable.

Dans ce livrable, le but est de traiter un ensemble d'images bruitées et d'améliorer leur qualité grâce à une technique de débruitage.
La méthode choisie repose sur l'utilisation d'autoencodeurs convolutionnels, un type d'architecture de réseau de neurones spécialement conçue pour le débruitage d'images.
Les autoencodeurs convolutionnels combinent des couches de convolution, qui sont efficaces pour préserver les relations spatiales dans les images, avec la capacité de l'autoencodeur à apprendre des représentations efficaces des données d'entrée.
Cette approche vise à supprimer le bruit tout en conservant les détails importants, ce qui donne des images de meilleure qualité pouvant améliorer la performance des algorithmes en aval.

## **Théorie : les DAE**

Le Denoising Autoencoder est une extension des autoencodeurs classiques, spécifiquement conçue pour améliorer la qualité des données en éliminant le bruit. Contrairement à un autoencodeur traditionnel qui apprend à reproduire ses entrées, le DAE est entraîné avec des images volontairement dégradées, sur lesquelles du bruit est ajouté. L'objectif est que le modèle apprenne à reconstruire l'image d'origine à partir de cette version altérée.

Cette approche présente plusieurs avantages majeurs. En premier lieu, elle empêche le réseau de se contenter de copier mécaniquement l'entrée, ce qui pourrait arriver avec un autoencodeur classique trop complexe. Ensuite, elle permet au modèle de se concentrer sur les caractéristiques importantes de l'image, en apprenant à distinguer les détails pertinents du bruit parasite. Enfin, cette stratégie renforce la robustesse du système face à des données de mauvaise qualité, ce qui est crucial dans le cadre du projet où les images brutes sont souvent bruitées ou de faible résolution.

Grâce au Denoising Autoencoder, les images sont nettoyées avant d'être utilisées par les étapes suivantes du pipeline, comme la classification ou la génération de légendes. Cela contribue à améliorer la précision globale du système de traitement d'images automatique.

![Architecture d'un DAE](DAE.jpeg)




Pour entraîner efficacement le Denoising Autoencoder, il est essentiel d'évaluer la qualité de la reconstruction de l'image propre à partir de sa version bruitée. Cela se fait à l'aide d'une fonction de perte (loss function), qui mesure l'écart entre l'image originale et l'image reconstruite.

Deux fonctions de perte sont couramment utilisées selon le format des données d'entrée :

Mean Squared Error (MSE) : utilisée lorsque les pixels des images sont exprimés par des valeurs continues (par exemple entre 0 et 1 ou 0 à 255). Cette fonction calcule la moyenne des carrés des écarts entre les pixels de l'image originale et ceux de l'image générée.

Binary Cross-Entropy (BCE) : préférable lorsque les images sont binarisées, c'est-à-dire que les pixels prennent uniquement les valeurs 0 ou 1. Cette fonction mesure la différence entre les distributions de pixels de l'entrée et de la sortie.

Le choix de la fonction de perte dépend donc du type de données utilisées, et joue un rôle crucial dans l'optimisation des performances du modèle de débruitage.

## 1. Prétraitement et préparation de l'environnement

In [ ]:
# Import des librairies
import os
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from collections import defaultdict
import shutil
import random
import cv2
from zipfile import ZipFile

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img
from sklearn.model_selection import train_test_split
from PIL import Image

# Affiche les graphiques Matplotlib directement dans le notebook Jupyter.
# %matplotlib inline

In [ ]:
# Extraction du dataset
def extract_dataset(zip_path, extract_to):
    """
    Extrait les fichiers d'un dataset zippé.

    Args:
        zip_path (str): Chemin vers le fichier zip.
        extract_to (str): Dossier où extraire les fichiers.
    """
    # Convertir les chemins en chemins absolus pour éviter les erreurs
    zip_path = os.path.abspath(zip_path)
    extract_to = os.path.abspath(extract_to)

    if not os.path.exists(zip_path):
        raise FileNotFoundError(f"Le fichier {zip_path} n'existe pas.")
    
    # Si le dossier de destination existe, le supprimer
    if os.path.exists(extract_to):
        shutil.rmtree(extract_to)
        print(f"Le dossier existant {extract_to} a été supprimé.")

    os.makedirs(extract_to, exist_ok=True)

    with ZipFile(zip_path, 'r') as zip_ref:
        # Aplatir la structure du zip : extraire les fichiers directement dans extract_to
        for member in zip_ref.namelist():
            filename = os.path.basename(member)
            # Vérifie si c'est un fichier zip contenant 'Livrable 1 - Photo' et 'Livrable 2' dans le nom
            if filename and filename.endswith('.zip') and any(keyword in filename for keyword in ['Livrable 1 - Photo', 'Livrable 2']):
                source = zip_ref.open(member)
                target_path = os.path.join(extract_to, filename)
                with source, open(target_path, "wb") as target:
                    target.write(source.read())
        

    # Extrait le contenu de tous les sous-zip **directement dans extract_to**
    for file in os.listdir(extract_to):
        if file.endswith('.zip'):
            sub_zip_path = os.path.join(extract_to, file)
            with ZipFile(sub_zip_path, 'r') as sub_zip_ref:
                for member in sub_zip_ref.namelist():
                    # Ne garde que le nom du fichier sans les sous-dossiers internes
                    filename = os.path.basename(member)
                    if filename:
                        source = sub_zip_ref.open(member)
                        target_file = os.path.join(extract_to, filename)
                        with source, open(target_file, "wb") as target:
                            target.write(source.read())
            os.remove(sub_zip_path)  # Supprime le zip après extraction
    
    print(f"Dataset extrait dans le dossier : {extract_to}")



Le dossier existant c:\Users\mallo\OneDrive\Bureau\CESI 2022 - 2025\Annee 5\Semestre 10\Option data science\Project\Data-science-project-A5\Dataset\Dataset2 a été supprimé.
Dataset extrait dans le dossier : c:\Users\mallo\OneDrive\Bureau\CESI 2022 - 2025\Annee 5\Semestre 10\Option data science\Project\Data-science-project-A5\Dataset\Dataset2


In [ ]:
# Appel de la fonction pour extraire le dataset
dataset_zip_path = './Dataset/Datasets.zip'
extract_dataset(dataset_zip_path, './Dataset/Dataset2')

In [ ]:
# Configuration
dataset_path = '/Dataset/Dataset2/'
img_size = (128, 128)  # Taille de l'image
num_samples = 6

# 1. Chargement et vérification des images
if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"Dossier introuvable : {dataset_path}")

def load_images(folder_path):
    """Charge toutes les images en mémoire et assure la cohérence des canaux"""
    images = []
    valid_ext = ('.png', '.jpg', '.jpeg')


    for f in os.listdir(folder_path):
        if f.lower().endswith(valid_ext):
            img = Image.open(os.path.join(folder_path, f))
            # Converti en RGB si necessaire pour assurer la coherence
            if img.mode != 'RGB':
                img = img.convert('RGB')
            images.append(np.array(img))

    if not images:
        raise ValueError("Aucune image valide trouvée")

    return images

images = load_images(dataset_path) # Charger les images d'origine

# Chargement des images du deuxième dataset
second_dataset_path = '/content/drive/My Drive/Projet/Dataset Livrable 1 - Photo/Photo'
Second_images = load_images(second_dataset_path)

# 2. Fonction pour obtenir les résolutions des images d'origine
def check_image_resolutions(folder):
    resolutions = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            resolutions.append(img.shape[:2])  # (hauteur, largeur)
    return resolutions

resolutions = check_image_resolutions(dataset_path)
heights = [res[0] for res in resolutions]
widths = [res[1] for res in resolutions]



# Affichage des résolutions d'origine
#print("Distribution des résolutions (hauteur, largeur) des images:")
#print(list(zip(heights, widths)))

# 3. Affichage de quelques images avec leurs résolutions d'origine
fig, axes = plt.subplots(1, num_samples, figsize=(15, 5))
for i in range(min(num_samples, len(images))):  # Sélectionner quelques images au hasard pour les afficher
    axes[i].imshow(images[i])
    axes[i].set_title(f" {heights[i]}x{widths[i]}")
    axes[i].axis('off')
plt.tight_layout()
plt.show()

## Redimensionnement du Dataset

Le fait que nos images aient différentes résolutions crée des défis pour l'entraînement d'un modèle de machine learning, notamment pour les réseaux de neurones convolutionnels (CNN), qui nécessitent des images uniformes. Voici pourquoi le redimensionnement est important :

1. **Uniformisation des données** : En redimensionnant toutes les images à une taille commune (par exemple, 128x128 ou 256x256), nous assurons que chaque image a le même nombre de pixels, facilitant ainsi l'apprentissage.

2. **Optimisation des performances** : Les résolutions élevées demandent plus de mémoire et ralentissent l'entraînement. Le redimensionnement permet de réduire la complexité tout en gardant l'essentiel des informations visuelles.

3. **Réduction des biais** : Des images de tailles différentes peuvent introduire un biais, où les plus grandes images sont privilégiées. Le redimensionnement garantit un traitement équitable de toutes les images.

4. **Simplification du prétraitement** : Le redimensionnement facilite également les étapes de prétraitement (comme l'ajout ou le débruitage) en garantissant des tailles uniformes.

Le choix de la taille de redimensionnement dépend du compromis entre la conservation des détails visuels et les exigences en ressources. Par exemple, 128x128 est adapté pour des projets légers, tandis que 256x256 conserve plus de détails mais consomme plus de mémoire et de calcul.

In [ ]:
# 4. Fonction pour Redimensionner les images
def resize_images(images, target_size=img_size):
    """Redimensionne toutes les images et les convertit en RGB"""
    resized_images = []
    for img in images:
        img_pil = Image.fromarray(img).convert('RGB')  # Forcer en RGB
        img_resized = img_pil.resize(target_size)
        resized_images.append(np.array(img_resized))
    return np.array(resized_images)



resized_images = resize_images(images, target_size=img_size)
resized_second_images = resize_images(Second_images, target_size=img_size)
# Sauvegarder pour les prochaines utilisations
#np.save('/content/drive/My Drive/Projet/second_images.npy', resized_second_images)
# Charger directement les images sauvegardées
#second_images = np.load('/content/drive/My Drive/Projet/second_images.npy', allow_pickle=True)


## Affichage d'échantillon de data correcte

In [ ]:

# 5. Affichage des images redimensionnées

all_images = np.concatenate([resized_images, resized_second_images], axis=0)
all_images = np.array(all_images)
print(f"Nombre d'images dans notre dataset : {len(all_images)}")


def display_images(images, num_samples=5):
    """Affiche un échantillon d'images"""
    plt.figure(figsize=(15, 5))
    for i in range(min(num_samples, len(all_images))):
        plt.subplot(1, num_samples, i+1)
        plt.imshow(images[i])
        plt.axis('off')
    plt.tight_layout()  # Ajout pour éviter le chevauchement
    plt.show()

# Appel de la fonction
display_images(all_images)





Dans cette partie, nous chargeons les images du dataset, vérifions leurs résolutions et les redimensionnons à une taille uniforme (256x256 pixels). Cela garantit que toutes les images ont les mêmes dimensions, ce qui est essentiel pour l’entraînement du modèle d'autoencodeur.

Nous pouvons également constaté que nous n'avons que 148 images dans notre Dataset ce qui est très peu pour notre modèle. C'est un risque de surapprentissage (overfitting). Avec un petit dataset, il est facile pour un modèle de surapprendre les données, c'est-à-dire de mémoriser les images spécifiques au lieu d'apprendre des motifs généraux.

Un petit dataset limite la quantité d'informations que le modèle peut exploiter, ce qui pourrait limiter ses performances finales, surtout sur des architectures complexes comme les autoencodeurs convolutionnels, qui ont beaucoup de paramètres à entraîner.

## Analyse des pixels et des couleurs des images

In [ ]:
#Analyse des pixels : calcul des statistiques

def calculate_image_statistics(all_images):
    """Calcule des statistiques sur les images"""
    pixel_values  = np.concatenate([img.ravel() for img in images]) #Rassemble tous les pixels de toutes les images en un seul tableau unidimensionnel
    mean = np.mean(pixel_values)
    std = np.std(pixel_values)
    median = np.median(pixel_values)

    return mean, std, median, pixel_values

mean, std, median, pixel_values = calculate_image_statistics(all_images)

#Affichage des statistiques
print(f"Moyenne : {mean}")
print(f"Écart type : {std}")
print(f"Médiane : {median}")

#Fonction pour visualiser la distribution des couleurs
def plot_color_distribution(all_images):
    """Affiche la distribution des couleurs des images"""
    reds = np.concatenate([img[:, :, 2].ravel() for img in images])
    greens = np.concatenate([img[:, :, 1].ravel() for img in images])
    blues = np.concatenate([img[:, :, 0].ravel() for img in images])

    plt.figure(figsize=(10, 5))
    plt.hist(reds, bins=50, color='red', alpha=0.5, label='Rouge') #extrait la composante rouge de chaque image, aplatit ces valeurs en un vecteur puis concatène tous ces vecteurs en un seul grand vecteur reds
    plt.hist(greens, bins=50, color='green', alpha=0.5, label='Vert')
    plt.hist(blues, bins=50, color='blue', alpha=0.5, label='Bleu')
    plt.title("Distribution des couleurs des images")
    plt.xlabel("Valeur des pixel (0-255)")
    plt.ylabel("Nombre de pixels")
    plt.legend()
    plt.show()






Dans notre jeu d'images, après avoir analysé les valeurs des pixels, voici les résultats obtenus :

Moyenne des pixels : 117.68

Écart-type des pixels : 71.70

Médiane des pixels : 115.00


Interprétation des résultats :
Moyenne des pixels : La moyenne est proche du centre de la plage [0, 255], indiquant que nos images ont une luminosité moyenne légèrement plus sombre que le milieu de l'échelle. Elles contiennent un bon équilibre entre zones sombres et claires, mais avec une légère prédominance de zones sombres.

Médiane des pixels : La médiane est très proche de la moyenne, ce qui montre que la distribution des pixels est relativement symétrique, sans valeurs extrêmes.

Écart-type : L'écart-type élevé (71.70) indique une grande variabilité des pixels, avec un bon contraste entre les zones très sombres et très claires, ce qui est utile pour les algorithmes de traitement d'image.


Nos images ont une luminosité légèrement inférieure à la moyenne, une distribution symétrique des valeurs de pixels et un bon contraste. Cela est idéal pour les phases de traitement d'image, car cela permet au modèle de mieux détecter les détails et les transitions dans les images.

In [ ]:
plot_color_distribution(all_images)

Contraste dans les Images :
Les pics aux extrêmes (0 et 1) dans la distribution des pixels indiquent un fort contraste dans les images, ce qui est important pour la netteté des bords et les zones de transition rapide. Lors du traitement, il est crucial de préserver ces zones pour ne pas altérer les détails fins et maintenir la clarté de l'image.

Variabilité des Couleurs :
Les différentes distributions entre les canaux (rouge, vert, bleu) montrent une bonne diversité des couleurs, bien que les teintes bleutées prédominent légèrement. Cela peut influencer la manière dont l'autoencodeur doit traiter chaque canal pour améliorer la qualité globale tout en respectant cette variation de couleurs.

Ajout de Bruit et Dégradation :
Lors de l'ajout de bruit, il est essentiel d'observer comment le bruit affecte les pics aux extrêmes et les zones intermédiaires. Lors du débruitage, l'autoencodeur doit être capable de reconstruire correctement ces zones de fort contraste et les couleurs dominantes.

## Data Augmentation

In [ ]:
# Nouvelle fonction de data augmentation
def augment_images(images, augment_factor=2): # Augmente le nombre d'images
    """Génère des images augmentées"""
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='reflect' #Les bords sont remplis par effet miroir : les valeurs de pixels sont copiées en sens inversé
   )

    augmented_images = []
    for img in images:
        img = img.reshape((1,) + img.shape)
        for _ in range(augment_factor):
            for batch in datagen.flow(img, batch_size=1):
                augmented_images.append(batch[0])
                break
    return np.array(augmented_images)

# Application avant le split
# print("Augmentation des données...")
# augmented_images = augment_images(all_images)
# all_images = np.concatenate([all_images, augmented_images])
# print(f"Nouveau nombre total d'images: {len(all_images)}")

In [ ]:
def preprocess_data(images):
    """Normalisation et redimensionnement"""
    images = images.astype('float32') / 255.  # Normalisation
    return images

# Split train/test
x_train, x_test = train_test_split(all_images, test_size=0.2, random_state=42)

# Application du prétraitement
x_train = preprocess_data(x_train)
x_test = preprocess_data(x_test)

print(f"\nForme des données :")
print(f"Train: {x_train.shape} (ex: {x_train[0].shape})")
print(f"Nombre d'images d'entrainement : {len(x_train) }")
print(f"Test: {x_test.shape} (ex: {x_test[0].shape})")
print(f"Nombre d'images test : {len(x_test) }")

In [ ]:
display_images(x_train, num_samples=6)

## Auto-encodeur de réduction de bruit (denoiser)

In [ ]:

def add_noise(images, noise_type="gaussian", noise_factor=0.3):
    noisy_images = images.copy()

    # if noise_type == "gaussian":
    #     noise = noise_factor * np.random.normal(loc=0.0, scale=1.0, size=images.shape)
    #     noisy_images += noise

    # elif noise_type == "poisson":
    #     noise = np.random.poisson(images * 255.0) / 255.0  # Convert to 0-1 range
    #     noisy_images += noise

    if noise_type == "speckle":
        noise = noise_factor * np.random.randn(*images.shape)
        noisy_images += images * noise

    # if noise_type == "salt_pepper":
    #     prob = noise_factor / 2
    #     rnd = np.random.rand(*images.shape)
    #     noisy_images[rnd < prob] = 0  # Sel (noir)
    #     noisy_images[rnd > (1 - prob)] = 1  # Poivre (blanc)

    return np.clip(noisy_images, 0., 1.)

# Appliquer plusieurs types de bruits
# x_train_noisy_gaussian = add_noise(x_train, "gaussian")
# x_test_noisy_gaussian = add_noise(x_test, "gaussian")

x_train_noisy_speckle = add_noise(x_train, "speckle")
x_test_noisy_speckle = add_noise(x_test, "speckle")

# x_train_noisy_sp = add_noise(x_train, "salt_pepper")
# x_test_noisy_sp = add_noise(x_test, "salt_pepper")

In [ ]:
display_images(x_train, num_samples=10)
# display_images(x_train_noisy_gaussian, num_samples=10)
display_images(x_train_noisy_speckle, num_samples=10)
# display_images(x_train_noisy_sp, num_samples=10)



In [ ]:
print("Dimensions des données :")
print("x_test shape:", x_test.shape)          # Doit être (N, 256, 256, 3)


In [ ]:
# Configurations principales de nos modèles
IMG_SIZE          = img_size                # taille coté final d'une image en pixel (ici 128x128)
NB_EPOCHS_DENOISE = 50               # nombre epoch alogithme debruiter
BATCH_SIZE        = 32               # taille batch de traitement
SAV_MODEL_DENOISE = "denoiser3.h5"     # sauvegarde du modele de debruitage

## Encodeur

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Input


# Data augmentation
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip('horizontal'),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1)
    ],
    name="data_augmentation"
)

# Entrée de l'image
input_img = Input(shape=(128, 128, 3))  # à adapter selon ta taille d’image

# Applique la data augmentation directement à l'entrée
x = data_augmentation(input_img)

# Bloc 1
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='same')(x)

# Bloc 2
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='same')(x)

# Bloc 3
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)



## Decodeur

In [ ]:
from tensorflow.keras.layers import UpSampling2D, Dropout, Conv2D, BatchNormalization

# Bloc 1
x = Conv2D(256, (3, 3), activation='relu', padding='same')(encoded)
x = Dropout(0.3)(x)  # Dropout un peu plus fort ici car c’est la couche la plus profonde
x = BatchNormalization()(x)
x = UpSampling2D((2, 2))(x)

# Bloc 2
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = UpSampling2D((2, 2))(x)

# Bloc 3
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = UpSampling2D((2, 2))(x)

# Dernière couche (sortie)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)


In [ ]:
from tensorflow.keras.models import Model

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse',  metrics=['accuracy'])  # MSE au lieu de binary_crossentropy pour les images continues
autoencoder.summary()


## Entrainement

In [ ]:
#%reload_ext tensorboard

In [ ]:

# history = autoencoder.fit(
    # x_train_noisy_gaussian, x_train,
    # epochs=50,
    # batch_size=64,
    # shuffle=True,
    # validation_data=(x_test_noisy_gaussian, x_test)
# )

history_speckle = autoencoder.fit(
    x_train_noisy_speckle, x_train,
    epochs=50,
    batch_size=64,
    shuffle=True,
    validation_data=(x_test_noisy_speckle, x_test)
)

# history_sp = autoencoder.fit(
#     x_train_noisy_sp, x_train,
#     epochs=50,
#     batch_size=32,
#     shuffle=True,
#     validation_data=(x_test_noisy_sp, x_test)
# )


In [ ]:
# Visualisation des pertes d'apprentissage (Train) et de validation (Test)
# plt.plot(history.history['loss'], label='train')  # Pertes d'apprentissage
# plt.plot(history.history['val_loss'], label='test')  # Pertes de validation
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Courbe d\'apprentissage')
plt.show()
plt.figure(figsize=(10, 5))

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Gaussian Noise
# axes[0].plot(history.history['loss'], label='Train')
# axes[0].plot(history.history['val_loss'], label='Test')
# axes[0].set_title("Gaussian Noise")
# axes[0].set_xlabel('Epochs')
# axes[0].set_ylabel('Loss')
# axes[0].legend()

# Speckle Noise
axes[1].plot(history_speckle.history['loss'], label='Train')
axes[1].plot(history_speckle.history['val_loss'], label='Test')
axes[1].set_title("Speckle Noise")
axes[1].set_xlabel('Epochs')
axes[1].legend()

# Salt & Pepper Noise
# axes[2].plot(history_sp.history['loss'], label='Train')
# axes[2].plot(history_sp.history['val_loss'], label='Test')
# axes[2].set_title("Salt & Pepper Noise")
# axes[2].set_xlabel('Epochs')
# axes[2].legend()

plt.tight_layout()
plt.show()


In [ ]:
#autoencoder.save(SAV_MODEL_DENOISE)  # Sauvegarde du modèle

In [ ]:
print("Dimensions des données :")
print("x_test shape:", x_test.shape)          # Doit être (N, 256, 256, 3)

print("Modèle input shape:", autoencoder.input_shape)  # Doit être (None, 256, 256, 3)


In [ ]:
def visualize_denoising(model, clean_images, noisy_images, num_samples=3):
    """Version corrigée avec affichage garanti"""
    # 1. Vérification des entrées
    num_samples = min(num_samples, len(clean_images))
    print(f"\nVisualisation de {num_samples} échantillons...")

    # 2. Prédiction
    try:
        denoised = model.predict(noisy_images[:num_samples], verbose=0)
        print("Prédiction réussie. Shapes :")
        print(f"- Original : {clean_images[0].shape}")
        print(f"- Bruité : {noisy_images[0].shape}")
        print(f"- Débruité : {denoised[0].shape}")
    except Exception as e:
        print("\nÉchec de prédiction :")
        print(f"Erreur : {str(e)}")
        print("Vérifiez que :")
        print("- Le modèle est compilé")
        print("- noisy_images.shape =", noisy_images.shape)
        print("- model.input_shape =", model.input_shape)
        return

    # 3. Configuration du plot
    plt.figure(figsize=(15, 8), dpi=100)  # Augmentation de la résolution

    # 4. Affichage des images
    for i in range(num_samples):
        # Calcul des métriques
        try:
            psnr_n = psnr(clean_images[i], noisy_images[i])
            psnr_d = psnr(clean_images[i], denoised[i])
        except:
            psnr_n = psnr_d = float('nan')  # En cas d'échec du calcul

        # Original
        plt.subplot(3, num_samples, i+1)
        plt.imshow(clean_images[i].squeeze(), cmap='gray' if clean_images[i].shape[-1] == 1 else None)
        plt.title(f"Original\nPSNR: ∞ dB", fontsize=8)
        plt.axis('off')

        # Bruité
        plt.subplot(3, num_samples, i+1+num_samples)
        plt.imshow(noisy_images[i].squeeze(), cmap='gray' if noisy_images[i].shape[-1] == 1 else None)
        plt.title(f"Noisy\nPSNR: {psnr_n:.2f} dB", fontsize=8)
        plt.axis('off')

        # Débruité
        plt.subplot(3, num_samples, i+1+2*num_samples)
        plt.imshow(denoised[i].squeeze(), cmap='gray' if denoised[i].shape[-1] == 1 else None)
        plt.title(f"Denoised\nPSNR: {psnr_d:.2f} dB", fontsize=8)
        plt.axis('off')

    # 5. Affichage final
    plt.tight_layout()
    plt.show()
    print("Visualisation terminée!")



In [ ]:
print("Dimensions des données :")
print("x_test shape:", x_test.shape)          # Doit être (N, 256, 256, 3)
# print("x_test_noisy shape:", x_test_noisy_gaussian.shape)  # Doit être (N, 256, 256, 3)
# print("x_test_noisy shape:", x_test_noisy_sp.shape)  # Doit être (N, 256, 256, 3)
print("x_test_noisy shape:", x_train_noisy_speckle.shape)  # Doit être (N, 256, 256, 3)

print("Modèle input shape:", autoencoder.input_shape)  # Doit être (None, 256, 256, 3)

In [ ]:
# # Test minimal
# print("=== Test de visualisation ===")
# print("x_test shape:", x_test.shape)          # Doit être (N, 256, 256, 3)
# # print("x_test_noisy shape:", x_test_noisy_gaussian.shape)  # Doit être (N, 256, 256, 3)
# # print("x_test_noisy shape:", x_test_noisy_sp.shape)  # Doit être (N, 256, 256, 3)
visualize_denoising(autoencoder, x_test[:3], x_train_noisy_speckle[:3])  # Test avec 3 échantillons
# visualize_denoising(autoencoder, x_test[:3], x_train_noisy_sp[:4])  # Test avec 3 échantillons